# Setup

In [10]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 


@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def llm(prompt, stop=["\n"]):
    #response = openai.Completion.create(
    response = openai.ChatCompletion.create(
      #model="text-davinci-002",
      #model="gpt-3.5-turbo",
      model="gpt-4",
      
      #prompt=prompt,
      messages = [{"role": "user", "content": prompt}],
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    #return response["choices"][0]["text"]
    return response['choices'][0]['message']['content']

# ReAct

In [11]:
# import wikipedia
dict_str_code_en = {
    'Thought': 'Thought', 
    'Action': 'Action',
    'Observation': 'Observation',
    'search': 'search',
    'lookup': 'lookup',
    'finish': 'finish',
    'instruction': """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
""",
    }

dict_str_code_es = {
    'Passage':'Pasaje', 
    'coherency score is': 'puntuación de coherencia es',
    'Choice': 'Elección',
    'more coherent passage is': 'pasaje más coherente es',
    'two passages are similarly coherent': 'dos pasajes son igualmente coherentes'
    }

dict_str_code_pt = {
    'Passage':'Passagem', 
    'coherency score is': 'pontuação de coerência é',
    'Choice': 'Escolha',
    'more coherent passage is': 'passagem mais coerente é',
    'two passages are similarly coherent': 'duas passagens são similarmente coerentes'
    }

dict_str_code_fr = {
    'Passage':'Passage', 
    'coherency score is': 'le score de cohérence est',
    'Choice': 'Choix',
    'search': 'search',
    'lookup': 'lookup'
    }

dict_str_code_it = {
    'Passage':'Passaggio', 
    'coherency score is': 'punteggio di coerenza è',
    'Choice': 'Scelta',
    'more coherent passage is': 'passaggio più coerente è',
    'two passages are similarly coherent': 'due passaggi sono similmente coerenti'
    }

dict_str_code = {
    'en': dict_str_code_en,
    'es': dict_str_code_es,
    'pt': dict_str_code_pt,
    'fr': dict_str_code_fr,
    'it': dict_str_code_it,
    }

In [12]:
def webthink(idx=None, to_print=True, lang=''):
    import wikienv, wrappers, requests
    env = wikienv.WikiEnv(lang=lang)
    env = wrappers.HotPotQAWrapper(env, split="dev")
    env = wrappers.LoggingWrapper(env)

    def step(env, action):
        attempts = 0
        while attempts < 10:
            try:
                return env.step(action)
            except requests.exceptions.Timeout:
                attempts += 1

    folder = './prompts/'
    prompt_file = f'prompts_naive_hotpot_{lang}.txt'
    with open(folder + prompt_file, 'r') as f:
        prompt = f.read()

    instruction = dict_str_code[lang]["instruction"]
    prompt = instruction + prompt

    Thought = dict_str_code[lang]["Thought"]
    Observation = dict_str_code[lang]["Observation"]
    Action = dict_str_code[lang]["Action"]
    finish = dict_str_code[lang]["finish"]

    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"{Thought} {i}:", stop=[f"\n{Observation} {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\n{Action} {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"{Thought} {i}: {thought}\n{Action} {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"{Thought} {i}: {thought}\n{Action} {i}: {action}\n{Observation} {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, f"{finish}[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [13]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

lang = 'en'
rs = []
infos = []
old_time = time.time()
#for i in idxs[:500]:
for i in idxs[:1]:
    r, info = webthink(i, to_print=True, lang=lang)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Question: What movie did actress Irene Jacob complete before the American action crime thriller film directed by Stuart Bird?
Thought 1: I need to search Irene Jacob and find the film she completed before the American action crime thriller film directed by Stuart Bird.
Action 1: Search[Irene Jacob]
Observation 1: Irène Marie Jacob (born 15 July 1966) is a French-Swiss actress known for her work with Polish film director Krzysztof Kieślowski.[2] She won the 1991 Cannes Film Festival Award for Best Actress for the Kieślowski film The Double Life of Veronique, and was nominated for the BAFTA Award for Best Actress in a Leading Role for her 1994 film Three Colours: Red. Her other film appearances include The Secret Garden (1993), Beyond the Clouds (1995), U.S. Marshals (1998), and Eternity (2016).. Irène Jacob was born in Suresnes, Hauts-de-Seine, a western suburb of Paris.[3] The youngest child with three older brothers, she was raised in a highly educated and intellectual family and

Traceback (most recent call last):
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/dc/yf3g734x4618v19csg17ygdw0000gn/T/ipykernel_54846/1585582134.py", line 12, in <module>
    r, info = webthink(i, to_print=True, lang=lang)
  File "/var/folders/dc/yf3g734x4618v19csg17ygdw0000gn/T/ipykernel_54846/1591708838.py", line 53, in webthink
    obs, r, done, info = step(env, f"{finish}[]")
  File "/var/folders/dc/yf3g734x4618v19csg17ygdw0000gn/T/ipykernel_54846/1591708838.py", line 11, in step
    return env.step(action)
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/react/wrappers.py", line 302, in step
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/react/wrappers.py", line 208, in step
    info.update(self.get_metrics(info))
  File "/Users/juglardiaz-zamora/Projec